<a href="https://colab.research.google.com/github/Aashi779/Efficient-multitasking-with-SLMs/blob/main/Task2_WebSearchSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Google Custom Search JSON API setup
SEARCH_API_KEY = ""
SEARCH_ENGINE_ID = "259848f7313264be6"
SEARCH_ENDPOINT = "https://www.googleapis.com/customsearch/v1"

# Function to perform a web search
def web_search(query):
    params = {
        "key": SEARCH_API_KEY,
        "cx": SEARCH_ENGINE_ID,
        "q": query,
        "num": 5
    }
    response = requests.get(SEARCH_ENDPOINT, params=params)
    response.raise_for_status()
    return response.json()

# Function to extract top web results
def get_top_search_results(query):
    search_results = web_search(query)
    top_results = search_results.get("items", [])
    return [result.get("snippet", "") for result in top_results if "snippet" in result]

# Load the Phi-3 model
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
phi3_model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

# Generate response from Phi-3 model
def generate_phi3_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = phi3_model.generate(**inputs, max_new_tokens=1024)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Summarize the search results
def summarize_search_results(snippets, query):
    string = """1. most relevant search result,
            2. second most relevant search result..."""
    prompt1 = f"Analyze the following {snippets} and rank them according to their relevance to the {query}. Give the output in the {string} format."
    result = generate_phi3_response(prompt1)

    prompt = f"Summarize the following search results:\n\n{result}\n\nSummary:"
    return generate_phi3_response(prompt)

# Web search workflow with latency calculation
def web_search_workflow(query):

    start_time = time.time()

    # Get search results
    search_snippets = get_top_search_results(query)


    search_latency = time.time() - start_time
    print(f"Web search latency: {search_latency:.2f} seconds")

    if not search_snippets:
        return "No relevant information found."


    start_summary_time = time.time()


    summary = summarize_search_results(search_snippets, query)


    summary_latency = time.time() - start_summary_time
    print(f"Summarization latency: {summary_latency:.2f} seconds")


    total_latency = time.time() - start_time
    print(f"Total workflow latency: {total_latency:.2f} seconds")

    return summary


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [ ]:

query = "What is machine learning and its types?"
result = web_search_workflow(query)
print(result)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Web search latency: 0.60 seconds
Summarization latency: 74.70 seconds
Total workflow latency: 75.30 seconds
Summarize the following search results:

Analyze the following ['Nov 29, 2023 ... There are primarily four types of machine learning: Supervised, Unsupervised, Semi-Supervised Learning and Reinforcement Learning.', 'Apr 3, 2024 ... What is machine learning? This field of study uses data and algorithms to mimic human learning, allowing machines to improve over time, becoming\xa0...', 'Apr 4, 2022 ... Machine learning (ML) is a discipline of artificial intelligence (AI) that provides machines with the ability to automatically learn from data and past\xa0...', 'In simpler terms, machine learning enables computers to learn from data and make decisions or predictions without being explicitly programmed to do so. At its\xa0...', 'Machine learning is a subset of AI, which enables the machine to automatically learn from data, improve performance from past experiences,\xa0...'] and rank t

In [ ]:
def calculate_rouge(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg=True)
    return scores

In [ ]:
reference_summary = """Machine learning is a type of artificial intelligence that enables computers to learn from data and improve their performance over time without being explicitly programmed."""
rouge_scores = calculate_rouge(reference_summary, result)
print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge-1': {'r': 0.96, 'p': 0.14285714285714285, 'f': 0.24870466095734114}, 'rouge-2': {'r': 0.7083333333333334, 'p': 0.06093189964157706, 'f': 0.1122112196634317}, 'rouge-l': {'r': 0.96, 'p': 0.14285714285714285, 'f': 0.24870466095734114}}


2nd Type

In [ ]:
!pip install transformers rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3800066a447a9efe381898b2564ad4c81054eb2b7cb4ac81f80aeaa83ec84fcd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install Rouge

In [ ]:
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge import Rouge

# Google Custom Search JSON API setup
SEARCH_API_KEY = ""
SEARCH_ENGINE_ID = "259848f7313264be6"
SEARCH_ENDPOINT = "https://www.googleapis.com/customsearch/v1"

# Model setup
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
phi3_model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

def web_search(query):
    params = {
        "key": SEARCH_API_KEY,
        "cx": SEARCH_ENGINE_ID,
        "q": query,
        "num": 5
    }
    response = requests.get(SEARCH_ENDPOINT, params=params)
    response.raise_for_status()
    return response.json()

def get_top_search_results(query, max_snippets=3, max_length=200):
    search_results = web_search(query)
    top_results = search_results.get("items", [])

    snippets = []
    for result in top_results[:max_snippets]:
        snippet = result.get("snippet", "")
        snippets.append(snippet[:max_length])

    return snippets

def generate_phi3_response(prompt, max_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = phi3_model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def summarize_search_results(snippets, query, max_snippets=3):
    truncated_snippets = snippets[:max_snippets]
    prompt = f"Summarize the following snippets for the query '{query}':\n\n" + " ".join(truncated_snippets)
    return generate_phi3_response(prompt, max_tokens=512)

def web_search_workflow(query):
    start_search_time = time.time()
    search_snippets = get_top_search_results(query)
    end_search_time = time.time()

    if not search_snippets:
        return "No relevant information found."

    start_summary_time = time.time()
    summary = summarize_search_results(search_snippets, query)
    end_summary_time = time.time()

    search_latency = end_search_time - start_search_time
    summarization_latency = end_summary_time - start_summary_time

    print(f"Web Search Latency: {search_latency:.2f} seconds")
    print(f"Summarization Latency: {summarization_latency:.2f} seconds")

    return summary

# ROUGE metrics evaluation
def calculate_rouge(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg=True)
    return scores


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query = "What is machine learning?"
result_summary = web_search_workflow(query)


Web Search Latency: 0.53 seconds
Summarization Latency: 79.20 seconds


In [ ]:
print("Generated Summary:", result_summary)


Generated Summary: Summarize the following snippets for the query 'What is machine learning?':

Machine learning (ML) is a branch of AI and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans ... Apr 21, 2021 ... Machine learning is a subfield of artificial intelligence, which is broadly defined as the capability of a machine to imitate intelligent human ... Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of ...

Machine learning is a subset of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.

The process of learning begins with observations or data, such as examples, direct experience, or instruction, in order to look for

In [ ]:
reference_summary = """Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable computers to improve their performance on a specific task through experience and data, without being explicitly programmed for that task. It is the process of training a computer system to automatically detect patterns in data and make decisions with minimal human intervention. In machine learning, the system learns by adjusting its internal parameters in response to the data it processes. This typically involves a training phase"""
rouge_scores = calculate_rouge(reference_summary, result_summary)
print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge-1': {'r': 0.7384615384615385, 'p': 0.2608695652173913, 'f': 0.38554216481669645}, 'rouge-2': {'r': 0.3176470588235294, 'p': 0.08157099697885196, 'f': 0.12980768905614376}, 'rouge-l': {'r': 0.7076923076923077, 'p': 0.25, 'f': 0.3694779077885841}}


Example-2

In [ ]:
query1 = "In the context of artificial intelligence, how can reinforcement learning be applied to optimize decision-making in autonomous systems, such as self-driving cars, while ensuring safety, ethical considerations, and adaptability to uncertain environments?"
result_summary1 = web_search_workflow(query1)

Web Search Latency: 0.40 seconds
Summarization Latency: 80.90 seconds


In [ ]:
print("Generated Summary:", result_summary1)

Generated Summary: Summarize the following snippets for the query 'In the context of artificial intelligence, how can reinforcement learning be applied to optimize decision-making in autonomous systems, such as self-driving cars, while ensuring safety, ethical considerations, and adaptability to uncertain environments?':

AI, ML, and DL are transforming the field of advanced robotics, making robots more intelligent, efficient, and adaptable to complex tasks and environments. Commercially envisioned and available self-driving cars can perform functions such ... Dogan, “Ethical decision making for autonomous vehicles,” in IEEE ... Other key terms defined in this chapter include intelligence and how this is used in the context of AI and intelligent robots (i.e. robots with an embedded AI), ...


Reinforcement learning (RL) is a type of machine learning where an agent learns to make decisions by taking actions in an environment to achieve a goal. The agent receives feedback in the form of 

In [ ]:
reference_summary1 = """Reinforcement learning (RL) is a type of machine learning that can be used to develop algorithms that allow autonomous systems, such as self-driving cars, to learn from their ongoing interactions with the environment. This methodology involves an agent that learns to make decisions by taking certain actions and receiving feedback, usually in the form of rewards or penalties. Here is how reinforcement learning can be applied to optimize decision-making while ensuring safety, eth"""
# Calculate ROUGE score against the reference summary
rouge_scores1 = calculate_rouge(reference_summary1, result_summary1)
print("ROUGE Scores:", rouge_scores1)

ROUGE Scores: {'rouge-1': {'r': 0.8032786885245902, 'p': 0.21212121212121213, 'f': 0.3356164350509008}, 'rouge-2': {'r': 0.5492957746478874, 'p': 0.10512129380053908, 'f': 0.17647058553868677}, 'rouge-l': {'r': 0.8032786885245902, 'p': 0.21212121212121213, 'f': 0.3356164350509008}}
